This notebook will load in the needed data from the milestoning simulation and save the cleaned data into a pair of tables for the milestoning coordinates and restraint parameters respectively.

If you are running this on google colab (or another cloud based site) you will need to clone the repository using the cell below (uncomment and run).

In [ ]:
#!git clone https://github.com/wesleymsmith/Milestoning_Analysis.git

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import tqdm
import copy
import gc
import sys
import os
import f90nml

import bokeh
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral11

from bokeh.models.mappers import CategoricalColorMapper

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import scipy as sp
from scipy.sparse import linalg

# Mulit-Replica MD milestoning simulation

Next, lets consider the case where some milestoning windows needed to be run in replicate. E.g. for some windows there are multiple individual replicates of the milestoning simulation that have been run (with different starting conformation / random seeds). This is useful, for instance, when it is found that too few edge crossings have occured over a given window edge and addtional replicates are run to increase sampling. Alternatively, this could be done to take advantage of having multiple gpus, workstations, cluster nodes, etc. wherein one would like to increase sampling by running multiple replicates rather than a single long simulation. I.e. cases where running additional simulations is more efficient than using additional resources on a single simulation.

In any case, care needs to be taken when collecting and analyzing milestoning data here because the individual replicas are not contiguous simulations but rather mimic an ensemble of simulations. For instane, if we treated replica simulations as contiguous, we could end up over counting / miscounting transition times where one replica ends and another starts. The solution is to analyze each replica individually and then agglomerate the results. Again, care must be taken here because we need to normalize over the total time spent in a given window, so we cannot simply take a raw mean of values over each replica. Thus, it is important to keep track of what data is coming from what replica in addition to each window.

For now, this simply means we need to add a column to our data to track replicas.

Since each replicate uses the same restraint setup, we need only modify how we store the simulation coordinates that are loaded.

In [2]:

dataDir='test_md_data/' #set this to the appropriate directory name
#dataDir='Milestoning_analysis/test_md_data' #uncomment this line if you needed to clone the repo
dataFiles=np.sort(os.listdir(dataDir))
dataFiles

array(['window_00.rest', 'window_00_rest.rep_01.step_10.dat',
       'window_00_rest.rep_01.step_11.dat', ...,
       'window_52_rest.rep_05.step_10.dat',
       'window_52_rest.rep_05.step_11.dat',
       'window_52_rest.rep_05.step_12.dat'], dtype='|S33')

The files ending in '.rest' are the restraint parameters for milestoning. They are formatted as fortran 90 namelists. The files ending in '.dat' are the ouput files for the simulation restraints and are in whitespace delimeted tables.

The column names for the '.dat' files are not given within the data file itself so we need to define them ourselves. The data file will contain some dummy columns which are present to make it more human readable.
We will label them accordingly so they can be easily removed later.

In addition to the reaction coordinate data from the '.dat' files, we also need to know about the parameters of the restraints that were used in each window. As mentioned before, this is found in the '.rest' files. These files contain only soft wall restraint settings.

These fortran 90 namelist formatted files can be loaded using the 'f90nml' package. We show an example in the cell below.

The relevant parameters are 'r2' and 'r3', which define the flat bottom portion of the milestoning restraint, and 'rk2' and 'rk3' which define the force constant of the pseudo-harmonic walls.

The data presented in the .dat files will correspond to the entries in the .rest files.

One important detail here is that each entry in the .rest file will tell us important information about how the data in the .dat file will be formatted.

The nmr restraint format used here is a bit too complex to go into fine detail. Interested readers should look in the appropriate section of the AMBER modeling and simulation manual.

In the case of more advanced restraint files, we would need to examine the 'iat' entry for each namelist entry to determine what kind of restraint we have.
For instance if there are 2 entries, we have a distance restraint. Angle type entries would have 3 iat entries and dihedral type entries would have 4. There can even be more than 4 entries in iat for various specialized restraint types...

For our purposes here, we only need to consider distance type restraints. In particular, our milestone windows are defined as 'flat bottom' harmonic potentials. The 'flat bottom' of these wells is what is important to us as that defines the region of our milestone window we need to sample from.

As such we can focus on a couple entry types. 

First, the entries r1 through r4 tell us about points of interest for our flat bottom potential. In particular, we want to know r2 and r3 which define the 'flat bottom' region.

Second, we need to look for the 'outxyz' flag. This is not so important for defining the restraint itself, but it will tell us important inforamtion about how our .dat file will be formatted. When set to 0 or absent, a single value corresponding to the measured distance will be output. When set to 1, additional information will be written detailing the x, y, and z coordinates of the restraint vector along with the total distance. Moreover, the data will contain 'x:', 'y:', and 'z:' in the columns preceding the x, y, and z data respectively. We will need to include appropriate dummy columns as placeholders.

Lets have a look at the first such file

In [3]:
restFiles=[dataFile for dataFile in dataFiles if '.rest' in dataFile]
tempRestFile=f90nml.read(dataDir+'/'+restFiles[0])
tempRestFile

Namelist([('rst',
           [Namelist([('iat', [65842, 65843]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                      ('rk2', 10.0),
                      ('rk3', 10.0)]),
            Namelist([('iat', [65842, 65846]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                      ('rk2', 10.0),
                      ('rk3', 10.0)]),
            Namelist([('iat', [65842, 65849]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                      ('rk2', 10.0),
                      ('rk3', 10.0)]),
            Namelist([('iat', [65842, 65852]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                  

Now we need to loop over each entry in this restraint parameter file and determine what the corresponding 'outxyz' entry is set to. If not explicitly present it can be assumed to be set to its default value of 1.

In [4]:
mainNamelistType='rst'
restXYZ=[rstDat['outxyz'] if 'outxyz' in rstDat else 0 for rstDat in tempRestFile[mainNamelistType]]
restXYZ

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

This simulation required a number of other restraints that are not relavent to the milestoning analysis. For instance, there are restraints to keep magnesium bound to key sites along the protein through which the ligand is being pulled so that the channel remains in its open state. For current purposes, we only need the Z coordinates for the waters-ligand restraints. They are labeled as 'W1L_Z' and 'W2L_Z'... More specifically, we only need 'W2L_Z' since this particular subset of the full milestoning data set only used water two to define its softwall restraints.

The cell below shows an example of what we will get upon reading in a '.dat' file

Previously we defined the needed columns by hand. Since there quite a few entries to track now, we need to automate this a bit.

In [5]:
datColNames=['Frame']
nRest=len(restXYZ)
nDigits=int(np.ceil(np.log10(nRest)))
baseNameStr='%s0%g%s'%('Rst%',nDigits,'g')
for iRst,outxyzVal in enumerate(restXYZ):
    if outxyzVal==0:
        datColNames=np.concatenate([
            datColNames,
            [baseNameStr%iRst]])
    else:
        datColNames=np.concatenate([
            datColNames,
            [(baseNameStr%iRst)+('_%s'%crd) for crd in [
                'DummyX','X','DummyY','Y','DummyZ','Z','R']]])
pd.read_csv(dataDir+'/'+dataFiles[1],delim_whitespace=True,
            names=datColNames).head()

,Frame,Rst00,Rst01,Rst02,Rst03,Rst04,Rst05,Rst06_DummyX,Rst06_X,Rst06_DummyY,...,Rst10_DummyZ,Rst10_Z,Rst10_R,Rst11_DummyX,Rst11_X,Rst11_DummyY,Rst11_Y,Rst11_DummyZ,Rst11_Z,Rst11_R
0,0,1.871,1.924,1.912,1.906,1.966,2.115,x:,4.291,y:,...,z:,52.082,53.404,x:,10.602,y:,5.200,z:,52.082,53.404
1,500,1.932,1.872,2.052,1.858,2.087,2.027,x:,4.117,y:,...,z:,52.069,53.389,x:,10.582,y:,5.218,z:,52.069,53.389
2,1000,1.939,1.984,1.955,1.906,1.885,1.942,x:,4.162,y:,...,z:,52.108,53.425,x:,10.584,y:,5.197,z:,52.108,53.425
3,1500,1.991,1.969,1.895,1.920,2.067,1.930,x:,4.621,y:,...,z:,52.130,53.454,x:,10.619,y:,5.204,z:,52.130,53.454
4,2000,1.959,1.973,1.908,2.010,1.972,1.828,x:,4.761,y:,...,z:,52.120,53.433,x:,10.570,y:,5.191,z:,52.120,53.433


In [6]:
#print windowDataTables

We are now ready to start loading in our data. In doing so, we will need to keep track of which window and replica each data file came from.

We will produce two combined tables. One for the '.dat' files and another for the '.rest' files.

Since our coordinate and restraint data contains a lot of uneeded information, we will use the 'restParmSubsetInds' variable to define a list of which restraint entries are needed from the restraint data files.
Additionally, we only need to know the values of certain sub-entries from each restraint.
Specifically we need 'r2' and 'r3' to locate the bounds of our milestone window. Additionally, we may want to know the force constant information at some point so we will retain 'rk2' and 'rk3' as well.

Similarly, we only need certain data columns from coordinate data as well.
These can be specified using the 'crdColumns' variable.

In [7]:
datFiles=[datFile for datFile in dataFiles if '.dat' in datFile]
restFiles=[restFile for restFile in dataFiles if '.rest' in restFile]
print 'reaction coordinate data files:',
print datFiles
print 'soft-wall restraint parameter files:',
print restFiles

windowDataTables=[]

restParmNMLname='rst' #This is the main namelist... probably don't need to change it

### MODIFY THESE VARIABLES AS NEEDED ### ###
#The next 3 variables should be changed to match the restraint and coordinate data
#setup you have
restParmNames=['r2','r3','rk2','rk3'] #select needed restraint sub-entry names
restParmSubsetInds=[7,8] #Select the needed restraint entries
crdColumns=['Rst06_X','Rst06_Y','Rst07_Z','Rst08_Z'] #Select the needed columns from the coordinate data
### ### ### ### ### ### ### ### ### ### ###

parmData={}
parmData['Window']=[]
for iParm,parmInd in enumerate(restParmSubsetInds):
    for parmName in restParmNames:
        parmData[baseNameStr%(parmInd)+'_%s'%parmName]=[]
        
for restFile in restFiles:
    print ('loading milestone restraint data from %s'%restFile)
    restFilePath=dataDir+'/'+restFile
    window=np.int(restFile.split('.')[0].split('_')[-1])
    tempNMLdata=f90nml.read(restFilePath)
    parmData['Window'].append(window)
    for iParm,restParmInd in enumerate(restParmSubsetInds):
        for restParmName in restParmNames:
            print restParmName
            parmData[baseNameStr%(restParmInd)+'_%s'%restParmName].append(tempNMLdata[
                restParmNMLname][restParmInd][restParmName])
windowRestraintTable=pd.DataFrame(parmData)
#print (windowRestraintTable.Window)
windowRestraintTable=windowRestraintTable[np.concatenate([
        ['Window'],
        [colName for colName in np.sort(parmData.keys()) \
         if colName!='Window']])]
print (windowRestraintTable)
windowRestraintTable.to_csv("Simulation_Milestone_Restraint_Data.MultiRepI_v2.csv",index=False)

reaction coordinate data files: ['window_00_rest.rep_01.step_10.dat', 'window_00_rest.rep_01.step_11.dat', 'window_00_rest.rep_01.step_12.dat', 'window_00_rest.rep_02.step_10.dat', 'window_00_rest.rep_02.step_11.dat', 'window_00_rest.rep_03.step_10.dat', 'window_00_rest.rep_03.step_11.dat', 'window_00_rest.rep_04.step_10.dat', 'window_00_rest.rep_04.step_11.dat', 'window_00_rest.rep_04.step_12.dat', 'window_00_rest.rep_05.step_10.dat', 'window_00_rest.rep_05.step_11.dat', 'window_00_rest.rep_05.step_12.dat', 'window_00_rest.rep_05.step_13.dat', 'window_00_rest.rep_06.step_10.dat', 'window_00_rest.rep_06.step_11.dat', 'window_01_rest.rep_01.step_10.dat', 'window_01_rest.rep_01.step_11.dat', 'window_01_rest.rep_01.step_12.dat', 'window_01_rest.rep_02.step_10.dat', 'window_01_rest.rep_02.step_11.dat', 'window_01_rest.rep_02.step_12.dat', 'window_01_rest.rep_03.step_10.dat', 'window_01_rest.rep_03.step_11.dat', 'window_01_rest.rep_03.step_12.dat', 'window_01_rest.rep_04.step_10.dat', 'wind


soft-wall restraint parameter files: ['window_00.rest', 'window_01.rest', 'window_02.rest', 'window_03.rest', 'window_04.rest', 'window_05.rest', 'window_06.rest', 'window_07.rest', 'window_08.rest', 'window_09.rest', 'window_10.rest', 'window_11.rest', 'window_12.rest', 'window_13.rest', 'window_14.rest', 'window_15.rest', 'window_16.rest', 'window_17.rest', 'window_18.rest', 'window_19.rest', 'window_20.rest', 'window_21.rest', 'window_22.rest', 'window_23.rest', 'window_24.rest', 'window_25.rest', 'window_26.rest', 'window_27.rest', 'window_28.rest', 'window_29.rest', 'window_30.rest', 'window_31.rest', 'window_32.rest', 'window_33.rest', 'window_34.rest', 'window_35.rest', 'window_36.rest', 'window_37.rest', 'window_38.rest', 'window_39.rest', 'window_40.rest', 'window_41.rest', 'window_42.rest', 'window_43.rest', 'window_44.rest', 'window_45.rest', 'window_46.rest', 'window_47.rest', 'window_48.rest', 'window_49.rest', 'window_50.rest', 'window_51.rest', 'window_52.rest']
loading

52        0.0        0.0  


In [8]:
for datFile in datFiles:
    print 'loading milestone coordinate data from %s'%datFile
    datFilePath=dataDir+'/'+datFile
    window=np.int(datFile.split('_')[1])
    if 'rep_' in datFile:
        rep=np.int(datFile.split('.')[1].split('_')[1])
    else:
        rep=1
    tempTable=pd.read_csv(datFilePath,delim_whitespace=True,
                          names=datColNames)
    tempTable['Window']=window
    tempTable['Rep']=rep
    windowTable=tempTable[['Window','Rep']].copy()
    windowTable['Time']=tempTable['Frame']
    for crdColumn in crdColumns:
        windowTable[crdColumn]=tempTable[crdColumn].abs()
    windowDataTables.append(windowTable.copy())
simData=pd.concat(windowDataTables)
print simData.head()    
simData.to_csv("Simulation_Milestone_Coordinate_Data.MultiRepI.csv",index=False)

loading milestone coordinate data from window_00_rest.rep_01.step_10.dat
loading milestone coordinate data from window_00_rest.rep_01.step_11.dat
loading milestone coordinate data from window_00_rest.rep_01.step_12.dat
loading milestone coordinate data from window_00_rest.rep_02.step_10.dat
loading milestone coordinate data from window_00_rest.rep_02.step_11.dat
loading milestone coordinate data from window_00_rest.rep_03.step_10.dat
loading milestone coordinate data from window_00_rest.rep_03.step_11.dat
loading milestone coordinate data from window_00_rest.rep_04.step_10.dat
loading milestone coordinate data from window_00_rest.rep_04.step_11.dat
loading milestone coordinate data from window_00_rest.rep_04.step_12.dat
loading milestone coordinate data from window_00_rest.rep_05.step_10.dat
loading milestone coordinate data from window_00_rest.rep_05.step_11.dat
loading milestone coordinate data from window_00_rest.rep_05.step_12.dat
loading milestone coordinate data from window_00_re

loading milestone coordinate data from window_04_rest.rep_06.step_13.dat
loading milestone coordinate data from window_04_rest.rep_07.step_10.dat
loading milestone coordinate data from window_04_rest.rep_07.step_11.dat
loading milestone coordinate data from window_04_rest.rep_07.step_12.dat
loading milestone coordinate data from window_04_rest.rep_07.step_13.dat
loading milestone coordinate data from window_04_rest.rep_07.step_14.dat
loading milestone coordinate data from window_04_rest.rep_07.step_15.dat
loading milestone coordinate data from window_04_rest.rep_07.step_16.dat
loading milestone coordinate data from window_04_rest.rep_07.step_17.dat
loading milestone coordinate data from window_04_rest.rep_08.step_10.dat
loading milestone coordinate data from window_04_rest.rep_08.step_11.dat
loading milestone coordinate data from window_04_rest.rep_08.step_12.dat
loading milestone coordinate data from window_04_rest.rep_08.step_13.dat
loading milestone coordinate data from window_04_re

loading milestone coordinate data from window_07_rest.rep_09.step_10.dat
loading milestone coordinate data from window_07_rest.rep_09.step_11.dat
loading milestone coordinate data from window_07_rest.rep_09.step_12.dat
loading milestone coordinate data from window_07_rest.rep_09.step_13.dat
loading milestone coordinate data from window_07_rest.rep_09.step_14.dat
loading milestone coordinate data from window_07_rest.rep_09.step_15.dat
loading milestone coordinate data from window_07_rest.rep_10.step_10.dat
loading milestone coordinate data from window_07_rest.rep_10.step_11.dat
loading milestone coordinate data from window_07_rest.rep_10.step_12.dat
loading milestone coordinate data from window_07_rest.rep_10.step_13.dat
loading milestone coordinate data from window_07_rest.rep_10.step_14.dat
loading milestone coordinate data from window_07_rest.rep_10.step_15.dat
loading milestone coordinate data from window_08_rest.rep_01.step_10.dat
loading milestone coordinate data from window_08_re

loading milestone coordinate data from window_09_rest.rep_08.step_10.dat
loading milestone coordinate data from window_09_rest.rep_08.step_11.dat
loading milestone coordinate data from window_09_rest.rep_08.step_12.dat
loading milestone coordinate data from window_09_rest.rep_08.step_13.dat
loading milestone coordinate data from window_09_rest.rep_08.step_14.dat
loading milestone coordinate data from window_09_rest.rep_08.step_15.dat
loading milestone coordinate data from window_09_rest.rep_08.step_16.dat
loading milestone coordinate data from window_09_rest.rep_08.step_17.dat
loading milestone coordinate data from window_09_rest.rep_08.step_18.dat
loading milestone coordinate data from window_09_rest.rep_08.step_19.dat
loading milestone coordinate data from window_09_rest.rep_08.step_20.dat
loading milestone coordinate data from window_09_rest.rep_08.step_21.dat
loading milestone coordinate data from window_09_rest.rep_08.step_22.dat
loading milestone coordinate data from window_09_re

loading milestone coordinate data from window_10_rest.rep_06.step_16.dat
loading milestone coordinate data from window_10_rest.rep_06.step_17.dat
loading milestone coordinate data from window_10_rest.rep_07.step_11.dat
loading milestone coordinate data from window_10_rest.rep_07.step_12.dat
loading milestone coordinate data from window_10_rest.rep_07.step_13.dat
loading milestone coordinate data from window_10_rest.rep_07.step_14.dat
loading milestone coordinate data from window_10_rest.rep_07.step_15.dat
loading milestone coordinate data from window_10_rest.rep_07.step_16.dat
loading milestone coordinate data from window_10_rest.rep_07.step_17.dat
loading milestone coordinate data from window_10_rest.rep_07.step_18.dat
loading milestone coordinate data from window_10_rest.rep_07.step_19.dat
loading milestone coordinate data from window_10_rest.rep_07.step_20.dat
loading milestone coordinate data from window_10_rest.rep_07.step_21.dat
loading milestone coordinate data from window_10_re

...

loading milestone coordinate data from window_24_rest.rep_03.step_27.dat
loading milestone coordinate data from window_24_rest.rep_03.step_28.dat
loading milestone coordinate data from window_24_rest.rep_03.step_29.dat
loading milestone coordinate data from window_24_rest.rep_03.step_30.dat
loading milestone coordinate data from window_25_rest.rep_01.step_10.dat
loading milestone coordinate data from window_25_rest.rep_01.step_11.dat
loading milestone coordinate data from window_25_rest.rep_01.step_12.dat
loading milestone coordinate data from window_25_rest.rep_01.step_13.dat
loading milestone coordinate data from window_25_rest.rep_01.step_14.dat
loading milestone coordinate data from window_25_rest.rep_01.step_15.dat
loading milestone coordinate data from window_25_rest.rep_01.step_16.dat
loading milestone coordinate data from window_25_rest.rep_01.step_17.dat
loading milestone coordinate data from window_26_rest.step_10.dat
loading milestone coordinate data from window_26_rest.step

loading milestone coordinate data from window_35_rest.rep_05.step_11.dat
loading milestone coordinate data from window_35_rest.rep_05.step_12.dat
loading milestone coordinate data from window_35_rest.rep_05.step_13.dat
loading milestone coordinate data from window_35_rest.rep_05.step_14.dat
loading milestone coordinate data from window_35_rest.rep_05.step_15.dat
loading milestone coordinate data from window_35_rest.rep_05.step_16.dat
loading milestone coordinate data from window_35_rest.rep_05.step_17.dat
loading milestone coordinate data from window_35_rest.rep_05.step_18.dat
loading milestone coordinate data from window_36_rest.step_10.dat
loading milestone coordinate data from window_36_rest.step_11.dat
loading milestone coordinate data from window_36_rest.step_12.dat
loading milestone coordinate data from window_36_rest.step_13.dat
loading milestone coordinate data from window_36_rest.step_14.dat
loading milestone coordinate data from window_36_rest.step_15.dat
loading milestone co

C:\Users\19094\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


loading milestone coordinate data from window_43_rest.rep_03.step_10.dat
loading milestone coordinate data from window_43_rest.rep_03.step_11.dat
loading milestone coordinate data from window_43_rest.rep_03.step_12.dat
loading milestone coordinate data from window_43_rest.rep_03.step_13.dat
loading milestone coordinate data from window_43_rest.rep_03.step_14.dat
loading milestone coordinate data from window_43_rest.rep_03.step_15.dat
loading milestone coordinate data from window_43_rest.rep_03.step_16.dat
loading milestone coordinate data from window_43_rest.rep_03.step_17.dat
loading milestone coordinate data from window_43_rest.rep_03.step_18.dat
loading milestone coordinate data from window_43_rest.rep_04.step_10.dat
loading milestone coordinate data from window_43_rest.rep_04.step_11.dat
loading milestone coordinate data from window_43_rest.rep_04.step_12.dat
loading milestone coordinate data from window_43_rest.rep_04.step_13.dat
loading milestone coordinate data from window_43_re

loading milestone coordinate data from window_49_rest.rep_02.step_12.dat
loading milestone coordinate data from window_49_rest.rep_02.step_13.dat
loading milestone coordinate data from window_49_rest.rep_03.step_10.dat
loading milestone coordinate data from window_50_rest.rep_01.step_10.dat
loading milestone coordinate data from window_50_rest.rep_01.step_11.dat
loading milestone coordinate data from window_50_rest.rep_01.step_12.dat
loading milestone coordinate data from window_50_rest.rep_01.step_13.dat
loading milestone coordinate data from window_50_rest.rep_03.step_10.dat
loading milestone coordinate data from window_50_rest.rep_03.step_11.dat
loading milestone coordinate data from window_50_rest.rep_04.step_10.dat
loading milestone coordinate data from window_50_rest.rep_04.step_11.dat
loading milestone coordinate data from window_51_rest.rep_01.step_10.dat
loading milestone coordinate data from window_51_rest.rep_01.step_11.dat
loading milestone coordinate data from window_51_re

Our simulation data is a bit more complex this time around since we have more replicas.

Also, as with the previous data set we have multiple coordinates tracked.
It would be nice to be able to automatically detect which coordinates we need to use.

We do so here by making use of the k values for each window. Any restraint which has a k-value of zero can essentially be ignored.

To make life a little easier on the analysis step to come, lets bin the coordinate data into the appropriate milestone windows here.

In order to do this, lets first extract the relevant window data from the restraint table.

One point of interest here is that the K values for each window will be important since they will tell us which coordinates were relevant for each window. This will become important shortly

In [9]:
binMins=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'r2' in colName]])
binMaxs=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'r3' in colName]])
binCenters=(binMins+binMaxs)/2
binRK2s=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'rk2' in colName]])
binRK3s=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'rk3' in colName]])
binKs=np.sqrt(binRK2s*binRK3s)
print 'binMins:',
print binMins
print 'binMaxs:',
print binMaxs
print 'binCenters:',
print binCenters
print 'binKs:'
print binKs

binMins: [[108.    9. ]
 [106.   11. ]
 [104.   13. ]
 [102.   15. ]
 [ 99.5  16.8]
 [ 97.5  18.8]
 [ 95.5  20.8]
 [ 93.5  22.8]
 [ 91.5  24.8]
 [ 89.5  26.8]
 [ 87.5  28.8]
 [ 85.5  30.8]
 [ 83.5  32.8]
 [ 81.5  34.8]
 [ 79.5  36.8]
 [ 77.5  38.8]
 [ 75.5  40.8]
 [ 73.5  42.8]
 [ 71.5  44.8]
 [ 69.5  46.8]
 [ 67.5  48.8]
 [ 65.5  50.8]
 [ 63.5  52.8]
 [ 61.5  54.8]
 [ 59.5  56.8]
 [ 57.5  58.8]
 [ 55.5  60.8]
 [ 53.5  62.8]
 [ 51.5  64.8]
 [ 49.5  66.8]
 [ 47.5  68.8]
 [ 45.5  70.8]
 [ 43.5  72.8]
 [ 41.5  74.8]
 [ 39.5  76.8]
 [ 37.5  78.8]
 [ 35.5  80.8]
 [ 33.5  82.8]
 [ 31.5  84.8]
 [ 29.5  86.8]
 [ 27.5  88.8]
 [ 25.5  90.8]
 [ 23.5  92.8]
 [ 21.5  94.8]
 [ 19.5  96.8]
 [ 17.5  98.8]
 [ 15.5 100.8]
 [ 13.5 102.8]
 [ 11.5 104.8]
 [  9.5 106.8]
 [  7.5 108.8]
 [  5.5 110.8]
 [  3.5 112.8]]
binMaxs: [[110.   11. ]
 [108.   13. ]
 [106.   15. ]
 [104.   17. ]
 [101.5  18.8]
 [ 99.5  20.8]
 [ 97.5  22.8]
 [ 95.5  24.8]
 [ 93.5  26.8]
 [ 91.5  28.8]
 [ 89.5  30.8]
 [ 87.5  32.8]
 [ 85.

Now we need to create a function that can assigne a given coordinate data point to the appropriate milestone window.

To do this, we need to know
    1. which window the coordinate came frome
    2. the bin centers for each window
    3. the k-values for each window
    4. the data coordinates
    
The first step is to select which restraint coordinates are relevant. This amounts to finding which restraint coordinates had non-zero K values.

From this we then subset our data coordinates and window centers, keeping only coordinates for which the restraint had a non-zero K value.

Finally we compute the distance from the remaining data coordinates using the corresponding coordinates of each window center. We then return the index of the window with the center that was closest to the data point.

In [10]:
#Function to compute bin index based on window and coordinate data
#X contains the coordinate data columns from the coordinate data table along
#with the 'Window' column
#e.g. x[0]=window, x[1] through x[n+1] are coordinates 0 through n
def binDataFunction(x,centers,kVals,verbose=False):
    X=np.array(x)
    if verbose:
        print 'x',
        print X
    coordInds=kVals[int(X[0])]>0 #only consider restraints with non-zero K values
    coordSet=X[1:][coordInds] #get subset of relevant data point coordinates
    centerSet=centers[:,coordInds] #get subset of relevant window center coordinates
    if verbose:
        print 'coordInds',
        print coordInds
        print 'coordSet',
        print coordSet
        print 'centerSet',
        print centerSet
        print 'outDists',
        print np.sqrt(np.sum(1.0*(centerSet-coordSet)**2,axis=1))
    #return the bin index of the center that is closest to the data point given
    return np.argmin(np.sqrt(np.sum(1.0*(centerSet-coordSet)**2,axis=1)))

In [11]:
print binCenters
print binKs
#print simData.X_Index.head()

[[109.   10. ]
 [107.   12. ]
 [105.   14. ]
 [103.   16. ]
 [100.5  17.8]
 [ 98.5  19.8]
 [ 96.5  21.8]
 [ 94.5  23.8]
 [ 92.5  25.8]
 [ 90.5  27.8]
 [ 88.5  29.8]
 [ 86.5  31.8]
 [ 84.5  33.8]
 [ 82.5  35.8]
 [ 80.5  37.8]
 [ 78.5  39.8]
 [ 76.5  41.8]
 [ 74.5  43.8]
 [ 72.5  45.8]
 [ 70.5  47.8]
 [ 68.5  49.8]
 [ 66.5  51.8]
 [ 64.5  53.8]
 [ 62.5  55.8]
 [ 60.5  57.8]
 [ 58.5  59.8]
 [ 56.5  61.8]
 [ 54.5  63.8]
 [ 52.5  65.8]
 [ 50.5  67.8]
 [ 48.5  69.8]
 [ 46.5  71.8]
 [ 44.5  73.8]
 [ 42.5  75.8]
 [ 40.5  77.8]
 [ 38.5  79.8]
 [ 36.5  81.8]
 [ 34.5  83.8]
 [ 32.5  85.8]
 [ 30.5  87.8]
 [ 28.5  89.8]
 [ 26.5  91.8]
 [ 24.5  93.8]
 [ 22.5  95.8]
 [ 20.5  97.8]
 [ 18.5  99.8]
 [ 16.5 101.8]
 [ 14.5 103.8]
 [ 12.5 105.8]
 [ 10.5 107.8]
 [  8.5 109.8]
 [  6.5 111.8]
 [  4.5 113.8]]
[[  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 10

In [12]:
#lets test the function
binDataFunction(np.array(simData[['Window','Rst07_Z','Rst08_Z']].iloc[31000]),
                binCenters,binKs,verbose=True)
#print simData.X_Index.head()


x [  0.    108.367   9.963]
coordInds [False  True]
coordSet [9.963]
centerSet [[ 10. ]
 [ 12. ]
 [ 14. ]
 [ 16. ]
 [ 17.8]
 [ 19.8]
 [ 21.8]
 [ 23.8]
 [ 25.8]
 [ 27.8]
 [ 29.8]
 [ 31.8]
 [ 33.8]
 [ 35.8]
 [ 37.8]
 [ 39.8]
 [ 41.8]
 [ 43.8]
 [ 45.8]
 [ 47.8]
 [ 49.8]
 [ 51.8]
 [ 53.8]
 [ 55.8]
 [ 57.8]
 [ 59.8]
 [ 61.8]
 [ 63.8]
 [ 65.8]
 [ 67.8]
 [ 69.8]
 [ 71.8]
 [ 73.8]
 [ 75.8]
 [ 77.8]
 [ 79.8]
 [ 81.8]
 [ 83.8]
 [ 85.8]
 [ 87.8]
 [ 89.8]
 [ 91.8]
 [ 93.8]
 [ 95.8]
 [ 97.8]
 [ 99.8]
 [101.8]
 [103.8]
 [105.8]
 [107.8]
 [109.8]
 [111.8]
 [113.8]]
outDists [3.70000e-02 2.03700e+00 4.03700e+00 6.03700e+00 7.83700e+00 9.83700e+00
 1.18370e+01 1.38370e+01 1.58370e+01 1.78370e+01 1.98370e+01 2.18370e+01
 2.38370e+01 2.58370e+01 2.78370e+01 2.98370e+01 3.18370e+01 3.38370e+01
 3.58370e+01 3.78370e+01 3.98370e+01 4.18370e+01 4.38370e+01 4.58370e+01
 4.78370e+01 4.98370e+01 5.18370e+01 5.38370e+01 5.58370e+01 5.78370e+01
 5.98370e+01 6.18370e+01 6.38370e+01 6.58370e+01 6.78370e+01 6.98370e

0

In [13]:
#binCenters=(binMins+binMaxs)/2
#binKs=np.sqrt(binRK2s*binRK3s)


#temp=[[116.5,1.8]]
#tempK=[[0.0,100.0]]
#for entry in binCenters:
#    temp.append(list(entry))
#for entry in binKs:
#    tempK.append(list(entry))
#binCenters=np.array(temp)
#binKs=np.array(tempK)

#temp1=[]
#for entry in binCenters:
#    temp1.append(list(entry))
#temp1.append([2.5,115.8])
#binCenters=np.array(temp1)

#tempK1=[]
#for entry in binKs:
#    tempK1.append(list(entry))
#tempK.append([100,0])
#binKs=np.array(tempK)


print binKs
binCenters

[[  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]
 [100.   0.]]


array([[109. ,  10. ],
       [107. ,  12. ],
       [105. ,  14. ],
       [103. ,  16. ],
       [100.5,  17.8],
       [ 98.5,  19.8],
       [ 96.5,  21.8],
       [ 94.5,  23.8],
       [ 92.5,  25.8],
       [ 90.5,  27.8],
       [ 88.5,  29.8],
       [ 86.5,  31.8],
       [ 84.5,  33.8],
       [ 82.5,  35.8],
       [ 80.5,  37.8],
       [ 78.5,  39.8],
       [ 76.5,  41.8],
       [ 74.5,  43.8],
       [ 72.5,  45.8],
       [ 70.5,  47.8],
       [ 68.5,  49.8],
       [ 66.5,  51.8],
       [ 64.5,  53.8],
       [ 62.5,  55.8],
       [ 60.5,  57.8],
       [ 58.5,  59.8],
       [ 56.5,  61.8],
       [ 54.5,  63.8],
       [ 52.5,  65.8],
       [ 50.5,  67.8],
       [ 48.5,  69.8],
       [ 46.5,  71.8],
       [ 44.5,  73.8],
       [ 42.5,  75.8],
       [ 40.5,  77.8],
       [ 38.5,  79.8],
       [ 36.5,  81.8],
       [ 34.5,  83.8],
       [ 32.5,  85.8],
       [ 30.5,  87.8],
       [ 28.5,  89.8],
       [ 26.5,  91.8],
       [ 24.5,  93.8],
       [ 22

In [14]:
print simData.head()

   Window  Rep    Time  Rst06_X  Rst06_Y  Rst07_Z  Rst08_Z
0       0    1     0.0    4.291   17.350  107.903   10.456
1       0    1   500.0    4.117   17.984  108.105   10.275
2       0    1  1000.0    4.162   17.821  108.509    9.857
3       0    1  1500.0    4.621   17.773  107.792   10.539
4       0    1  2000.0    4.761   17.909  108.495    9.874


In [15]:
#tqdm.tqdm_pandas(tqdm.tqdm_notebook())
#tqdm.tqdm_pandas(tqdm.tqdm_notebook())


We are now ready to bin our simulation restraint coordinate data.

We will need the 'Window' column along with all columns that had 'Rst' in the column name.

We will feed the window centers and k values we constructed above using a keyword dictionary which we will supply to the pandas 'apply' command.

In [16]:
#binKwds={
   # 'centers':binCenters,
    #'kVals':binKs,
   # #'verbose':True
#}
#simData['Window']=simData.Window+1
#simData['X_Index']=simData[np.concatenate([
   # ['Window'],
    #[colName for colName in simData.columns if 'Rst' in colName]
#])].apply(binDataFunction,axis=1,**binKwds)
#simData.head()
tqdm.tqdm_pandas(tqdm.tqdm_notebook())
binKwds={
    'centers':binCenters,
    'kVals':binKs,
    #'verbose':True
}
simData['Window']=simData.Window
simData['X_Index']=simData[np.concatenate([
    ['Window'],
    [colName for colName in simData.columns if ('Rst07' in colName) and ('Z' in colName)],
    [colName for colName in simData.columns if ('Rst08' in colName) and ('Z' in colName)]
])].apply(binDataFunction,axis=1,**binKwds)
simData.head()


,Window,Rep,Time,Rst06_X,Rst06_Y,Rst07_Z,Rst08_Z,X_Index
0,0,1,0.0,4.291,17.350,107.903,10.456,0
1,0,1,500.0,4.117,17.984,108.105,10.275,0
2,0,1,1000.0,4.162,17.821,108.509,9.857,0
3,0,1,1500.0,4.621,17.773,107.792,10.539,0
4,0,1,2000.0,4.761,17.909,108.495,9.874,0


As a quick check, lets make sure we see transitions between adjacent windows.

We can do this using the pandas groupby-aggregate method and serializing the list of unique X_Index values seen for each window

In [17]:
simData.groupby(['Window']).agg({'X_Index':lambda x: ','.join([str(val) for val in np.unique(x)])})

,X_Index
Window,
0,"0,1"
1,"0,1,2"
2,"1,2,3"
3,"2,3,4"
4,"3,4,5"
5,"4,5,6"
6,"5,6,7"
7,"6,7,8"
8,"7,8,9"


Now lets re-export this data so that we can have the X_Index available during analysis.

In [18]:
#simData.describe()

In [19]:
simData.to_csv("Simulation_Milestone_Coordinate_Data.MultiRepI.csv",index=False)


In [55]:
print simData.head()

   Window  Rep    Time  Rst06_X  Rst06_Y  Rst07_Z  Rst08_Z  X_Index
0       0    1     0.0    4.291   17.350  107.903   10.456        0
1       0    1   500.0    4.117   17.984  108.105   10.275        0
2       0    1  1000.0    4.162   17.821  108.509    9.857        0
3       0    1  1500.0    4.621   17.773  107.792   10.539        0
4       0    1  2000.0    4.761   17.909  108.495    9.874        0


In [34]:
print (np.sort(parmData.keys()))

['Rst07_r2' 'Rst07_r3' 'Rst07_rk2' 'Rst07_rk3' 'Rst08_r2' 'Rst08_r3'
 'Rst08_rk2' 'Rst08_rk3' 'Window']
